## Homework 5

In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split 


from unbalanced_dataset import UnderSampler, OverSampler, SMOTE


In [87]:
cancers= pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data',header=None)
cancers.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [88]:
cancers.columns=['id','Clump','Uni.Cell Size','Uni.Cell Shape','Marg. Adhesion','SE Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class']

In [89]:
cancers.head()

,id,Clump,Uni.Cell Size,Uni.Cell Shape,Marg. Adhesion,SE Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [90]:
cancers1 = cancers[cancers['Bare Nuclei']!='?']

In [91]:
cancers1['Class'].value_counts()

2    444
4    239
Name: Class, dtype: int64

In [92]:
X = cancers1.drop('Class',axis=1)
y= cancers1['Class']

In [93]:
x = X.values
y = y.values

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2)  

In [94]:
len(Y_train[Y_train == 2])

352

In [95]:
US = UnderSampler(verbose=True)
usx, usy = US.fit_transform(X_train,Y_train)

Determining classes statistics... 2 classes detected: {2: 352, 4: 194}
Under-sampling performed: Counter({2: 194, 4: 194})


In [96]:
us_df = pd.DataFrame(usx, columns=X.columns)
us_df['labels'] = usy
us_df.head()

,id,Clump,Uni.Cell Size,Uni.Cell Shape,Marg. Adhesion,SE Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,labels
0,1107684,6,10,5,5,4,10,6,10,1,4
1,1108370,9,5,8,1,2,3,2,1,5,4
2,1198128,10,8,10,10,6,1,3,1,10,4
3,832567,4,2,3,5,3,8,7,6,1,4
4,274137,8,8,9,4,5,10,7,8,1,4


In [97]:
us_df= us_df.drop('id',axis=1)

In [98]:
X1 = us_df.drop('labels',axis=1)
y1 = us_df['labels']

In [106]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_norm = scaler.fit_transform(X1)

X1_df = pd.DataFrame(X_norm, columns=X1.columns)

In [107]:
x2 = X1_df.values
y2 = y1.values

X_train, X_test, Y_train, Y_test = train_test_split(x2, y2, test_size=0.2)  

In [108]:
model = SVC(kernel='linear',C=1).fit(X_train,Y_train)
print classification_report(Y_test,model.predict(X_test))


             precision    recall  f1-score   support

          2       0.97      0.97      0.97        40
          4       0.97      0.97      0.97        38

avg / total       0.97      0.97      0.97        78



In [110]:
model = SVC(kernel='linear',C=1)
model.fit(X_train, Y_train)  
predictions = model.predict(X_test)

cm = confusion_matrix(Y_test,predictions)

#lets make that easier to read
cm_df = pd.DataFrame(cm, columns=['Predicted Class 2', 'Predicted Class 4'], 
                         index=['Actual Class 2', 'Actual Class 4'])

cm_df

,Predicted Class 2,Predicted Class 4
Actual Class 2,39,1
Actual Class 4,1,37


In [126]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def plot_roc_curve(target_test, target_predicted):
    fpr, tpr, thresholds = roc_curve(target_test, target_predicted[:, 1])
    
    roc_auc = auc(fpr, tpr)
    
    p = figure(title='Receiver Operating Characteristic')
    # Plot ROC curve
    p.line(x=fpr,y=tpr,legend='ROC curve (area = %0.3f)' % roc_auc)
    p.x_range=Range1d(0,1)
    p.y_range=Range1d(0,1)
    p.xaxis.axis_label='False Positive Rate or (1 - Specifity)'
    p.yaxis.axis_label='True Positive Rate or (Sensitivity)'
    p.legend.orientation = "bottom_right"
    show(p)

In [127]:
model = SVC(kernel='linear',C=1).fit(X_train,Y_train)
target_predicted = model.predict(X_test)

Sorry I ran out of time to complete the homework assignment this week

In [131]:
from sklearn.tree import DecisionTreeRegressor
treereg = DecisionTreeRegressor(random_state=1)
treereg.fit(X_train, Y_train)
print classification_report(Y_test,treereg.predict(X_test))

             precision    recall  f1-score   support

          2       0.95      0.97      0.96        40
          4       0.97      0.95      0.96        38

avg / total       0.96      0.96      0.96        78



In [132]:
treereg = DecisionTreeRegressor(random_state=1)
treereg.fit(X_train, Y_train)  
predictions = model.predict(X_test)

cm = confusion_matrix(Y_test,predictions)

#lets make that easier to read
cm_df = pd.DataFrame(cm, columns=['Predicted Class 2', 'Predicted Class 4'], 
                         index=['Actual Class 2', 'Actual Class 4'])

cm_df

,Predicted Class 2,Predicted Class 4
Actual Class 2,39,1
Actual Class 4,1,37


In [133]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
randfor =RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
randfor.fit(X_train,Y_train)
print classification_report(Y_test,randfor.predict(X_test))


             precision    recall  f1-score   support

          2       0.97      0.97      0.97        40
          4       0.97      0.97      0.97        38

avg / total       0.97      0.97      0.97        78



In [134]:
randfor =RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
randfor.fit(X_train,Y_train)
predictions = model.predict(X_test)

cm = confusion_matrix(Y_test,predictions)

#lets make that easier to read
cm_df = pd.DataFrame(cm, columns=['Predicted Class 2', 'Predicted Class 4'], 
                         index=['Actual Class 2', 'Actual Class 4'])

cm_df

,Predicted Class 2,Predicted Class 4
Actual Class 2,39,1
Actual Class 4,1,37


Sorry ran out of time to complete homework assignment